# Singleton Networks

In [1]:
import qualreas as qr
import os
import copy

In [2]:
qr_path = os.path.join(os.getenv('PYPROJ'), 'qualreas')
alg_dir = os.path.join(qr_path, "Algebras")

## Make a Test Network

In [50]:
test1_net_dict = {
    'name': 'Network Copy Test #1',
    'algebra': 'Extended_Linear_Interval_Algebra',
    'description': 'Testing/Developing network copy functionality',
    'nodes': [
        ['U', ['ProperInterval', 'Point']],
        ['V', ['ProperInterval', 'Point']],
        ['W', ['ProperInterval']],
        ['X', ['Point']]
    ],
    'edges': [
        ['U', 'V', 'B'],
        ['U', 'W', 'M'],
        ['W', 'V', 'O'],
        ['X', 'W', 'D']
    ]
}

In [51]:
test1_net = qr.Network(algebra_path=alg_dir, network_dict=test1_net_dict)

In [52]:
test1_net.propagate()
test1_net.summary()


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: M
    => X: B
  V:['ProperInterval']
    => V: E
    => U: BI
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => U: MI
    => V: O
    => X: DI
  X:['Point']
    => X: PE
    => W: D
    => U: BI
    => V: B|D|PS


### Test Short Version of Network Summary

In [53]:
test1_net.summary(show_all=False)


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: M
    => X: B
  V:['ProperInterval']
    => V: E
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => X: DI
  X:['Point']
    => X: PE


## Test Changing Constraint on an Edge

Look at all the edge contraints

In [54]:
for eg in test1_net.edges:
    print(test1_net.edges[eg[0], eg[1]]['constraint'])

E
B
M
B
E
BI
OI
BI|DI|PSI
E
MI
O
DI
PE
D
BI
B|D|PS


Grab the Head (src) and Tail (tgt) of the 3rd edge, above.

In [55]:
src, tgt = list(test1_net.edges)[2]
test1_net.edges[src,tgt]['constraint']

relset(['M'])

Change the constraint and look at the result on the edge & its converse.

In [56]:
test1_net.set_constraint(src, tgt, test1_net.algebra.relset('D|M|FI'))

In [57]:
test1_net.edges[src,tgt]['constraint']

relset(['D', 'FI', 'M'])

In [58]:
test1_net.edges[tgt,src]['constraint']

relset(['DI', 'F', 'MI'])

## Test Copy Network

In [59]:
test1_net_copy = test1_net.copy()
#test1_net_copy = qr.copy(test1_net)

In [60]:
test1_net_copy.summary()


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: D|FI|M
    => X: B
  V:['ProperInterval']
    => V: E
    => U: BI
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => U: DI|F|MI
    => V: O
    => X: DI
  X:['Point']
    => X: PE
    => U: BI
    => V: B|D|PS
    => W: D


In [70]:
done = []
result = []
for eg in test1_net.edges:
    src = eg[0]; tgt = eg[1]
    srcID = src.name; tgtID = tgt.name
    if not (src, tgt) in done:
        cons = test1_net.edges[src, tgt]['constraint']
        print(srcID, tgtID, cons)
        if len(cons) > 1:
            result.append((srcID, tgtID, cons))
    done.append((tgt, src))

U U E
U V B
U W D|FI|M
U X B
V V E
V W OI
V X BI|DI|PSI
W W E
W X DI
X X PE


In [73]:
rels = []
for rel in result[0][2]:
    rels.append(rel)

In [74]:
rels

['D', 'FI', 'M']